<a href="https://colab.research.google.com/github/andryll/MGR-IC/blob/main/codes/Feature_Extraction_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importando as Bibliotecas

import numpy as np
import librosa
#import IPython.display as ipd
import os
import pandas as pd
from sklearn import preprocessing
from sklearn.decomposition import PCA

In [2]:
def splitSongs (songList, duration, sr=44100):

  # Converta o tamanho da janela de segundos para amostras
  window_size_samples = int(duration * sr)

  # Inicialize uma lista para armazenar os segmentos
  segmentedList = []

  # Pega cada música da lista
  for y in songList:

    # Calcule o número total de segmentos
    num_segments = len(y[0]) // window_size_samples

    # Divida o áudio em segmentos de 5 segundos e adcione-os na lista
    for i in range(num_segments):
        start = i * window_size_samples
        end = (i + 1) * window_size_samples
        segment = (y[0][start:end], y[1], y[2])
        segmentedList.append(segment)

  # Retorna a nova lista
  return segmentedList

In [3]:
def readSongs (genre, numSongs, sr=44100, duration = 30):

  # Declarando listas iniciais
  songs = []
  genrelist = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz',
                'metal', 'pop', 'reggae', 'rock']

  # Se a escolha de gênero não for 'all', substitui a lista pelo gênero escolhido
  if genre != 'all':
    genrelist = [genre]

  #Percorre todos os gêneros da lista
  for g in genrelist:
    #Pega o caminho para a pasta do gênero escolhido
    dir_path = os.path.join('C:/Users/andry/OneDrive/Documentos/GitHub/MGR-IC/songs', g)
    #Lista os arquivos da pasta e os embaralha
    files = os.listdir(dir_path)
    files.sort

    # Até o número de musicas desejado ser alcançado, lê os arquivos de áudio com o librosa
    for i in range(numSongs):
      songs.append(librosa.load(os.path.join(dir_path, files[i]), sr=sr, mono = True, duration = 30))
      # Adciona o gênero como uma variável da tupla
      songs[-1] = songs[-1] + (g,)

  max_len = max(len(song[0]) for song in songs)

  # Garante que todas as músicas terão o mesmo tamanho da maior
  resized_songs = []
  for song in songs:
      # Verifica se a música precisa ser redimensionada
      if len(song[0]) < max_len:
          # Adiciona zeros à direita para igualar o tamanho
          padded_audio = librosa.util.pad_center(data = song[0], size = max_len, axis = 0)
          resized_songs.append((padded_audio, song[1], song[2]))
      else:
          resized_songs.append(song)

  new_songs = splitSongs (resized_songs, sr=sr, duration = duration)

  return new_songs

### Extração das Features


In [4]:
def featureExtraction (songs, sr=44100, frame=512):

  colunas = ['Chroma mean C', 'Chroma sd2 C','Chroma mean C#', 'Chroma sd2 C#', 'Chroma mean D', 'Chroma sd2 D', 'Chroma mean D#', 'Chroma sd2 D#',
             'Chroma mean E', 'Chroma sd2 E', 'Chroma mean F', 'Chroma sd2 F', 'Chroma mean F#', 'Chroma sd2 F#', 'Chroma mean G', 'Chroma sd2 G',
             'Chroma mean Ab', 'Chroma sd2 Ab', 'Chroma mean A', 'Chroma sd2 A', 'Chroma mean Bb', 'Chroma sd2 Bb', 'Chroma mean B', 'Chroma sd2 B',
             'RMS mean', 'RMS sd2', 'SpC mean', 'SpC sd2', 'SpBw mean', 'SpBw sd2', 'SpR99 mean', 'SpR99 sd2', 'SpR01 mean', 'SpR01 sd2', 'ZCR mean',
             'ZCR sd2', 'MFCC1 mean', 'MFCC1 sd2', 'MFCC2 mean', 'MFCC2 sd2', 'MFCC3 mean', 'MFCC3 sd2', 'MFCC4 mean', 'MFCC4 sd2', 'MFCC5 mean',
             'MFCC5 sd2', 'MFCC6 mean', 'MFCC6 sd2', 'MFCC7 mean', 'MFCC7 sd2', 'MFCC8 mean', 'MFCC8 sd2', 'MFCC9 mean', 'MFCC9 sd2', 'MFCC10 mean',
             'MFCC10 sd2', 'MFCC11 mean', 'MFCC11 sd2', 'MFCC12 mean', 'MFCC12 sd2', 'MFCC13 mean', 'MFCC13 sd2', 'MFCC14 mean', 'MFCC14 sd2', 'MFCC15 mean',
             'MFCC15 sd2', 'MFCC16 mean', 'MFCC16 sd2', 'MFCC17 mean', 'MFCC17 sd2', 'MFCC18 mean', 'MFCC18 sd2', 'MFCC19 mean', 'MFCC19 sd2', 'MFCC20 mean',
             'MFCC20 sd2', 'tempo', 'Classe']

  df = pd.DataFrame(columns=colunas)

  for y in songs:

    #Chroma
    chromas = librosa.feature.chroma_stft(y=y[0], sr=y[1])
    chroma_mean = np.mean(chromas, axis=1)
    chroma_sd2 = np.std(chromas, axis=1)**2

    #RMS
    rms = librosa.feature.rms(y=y[0], frame_length=frame, hop_length=int(frame/2))
    rms_mean = np.mean(rms)
    rms_sd2 = np.std(rms)**2

    #Spectral Centroid
    centroid = librosa.feature.spectral_centroid(y=y[0], sr=y[1], hop_length=int(frame/2), n_fft=frame)
    centroid_mean = np.mean(centroid)
    centroid_sd2 = np.std(centroid)**2

    #Spectral Bandwidth
    bandwidth = librosa.feature.spectral_bandwidth(y=y[0], sr = y[1], n_fft=frame, hop_length=int(frame/2))
    bandwidth_mean = np.mean(bandwidth)
    bandwidth_sd2 = np.std(bandwidth)**2

    #Spectral Rolloff
    roll99 = librosa.feature.spectral_rolloff(y=y[0], sr=y[1], n_fft=frame, hop_length=int(frame/2), roll_percent=0.99)
    roll99_mean = np.mean(roll99)
    roll99_sd2 = np.std(roll99)**2

    roll01 = librosa.feature.spectral_rolloff(y=y[0], sr=y[1], n_fft=frame, hop_length=int(frame/2), roll_percent=0.01)
    roll01_mean = np.mean(roll01)
    roll01_sd2 = np.std(roll01)**2

    #ZCR
    zcr = librosa.feature.zero_crossing_rate(y=y[0], frame_length=frame, hop_length=int(frame/2))
    zcr_mean = np.mean(zcr)
    zcr_sd2 = np.std(zcr)**2

    #MFCC
    mfcc = librosa.feature.mfcc(y=y[0], sr=y[1], n_mfcc=20)
    mfcc_mean = np.mean(mfcc, axis=1)
    mfcc_sd2 = np.std(mfcc, axis=1)**2
    #Harmony

    #tempo
    tempo = librosa.feature.tempo(y=y[0], sr=y[1], hop_length=int(frame/2))

    #Adcionando ao DF

    df.loc[len(df)] = [chroma_mean[0], chroma_sd2[0], chroma_mean[1], chroma_sd2[1], chroma_mean[2], chroma_sd2[2], chroma_mean[3], chroma_sd2[3],
                      chroma_mean[4], chroma_sd2[4], chroma_mean[5], chroma_sd2[5], chroma_mean[6], chroma_sd2[6], chroma_mean[7], chroma_sd2[7],
                      chroma_mean[8], chroma_sd2[8], chroma_mean[9], chroma_sd2[9], chroma_mean[10], chroma_sd2[10], chroma_mean[11], chroma_sd2[11],
                      rms_mean, rms_sd2, centroid_mean, centroid_sd2, bandwidth_mean, bandwidth_sd2, roll99_mean, roll99_sd2, roll01_mean, roll01_sd2,
                      zcr_mean, zcr_sd2, mfcc_mean[0], mfcc_sd2[0], mfcc_mean[1], mfcc_sd2[1], mfcc_mean[2], mfcc_sd2[2], mfcc_mean[3], mfcc_sd2[3],
                      mfcc_mean[4], mfcc_sd2[4], mfcc_mean[5], mfcc_sd2[5], mfcc_mean[6], mfcc_sd2[6], mfcc_mean[7], mfcc_sd2[7], mfcc_mean[8], mfcc_sd2[8],
                      mfcc_mean[9], mfcc_sd2[9], mfcc_mean[10], mfcc_sd2[10], mfcc_mean[11], mfcc_sd2[11], mfcc_mean[12], mfcc_sd2[12], mfcc_mean[13], mfcc_sd2[13],
                      mfcc_mean[14], mfcc_sd2[14], mfcc_mean[15], mfcc_sd2[15], mfcc_mean[16], mfcc_sd2[16], mfcc_mean[17], mfcc_sd2[17], mfcc_mean[18], mfcc_sd2[18],
                      mfcc_mean[19], mfcc_sd2[19], tempo[0], y[2]]

    # df = df.append({colunas[0]: chroma_mean[0], colunas[1]: chroma_sd2[0], colunas[2]: chroma_mean[0], colunas[3]: chroma_sd2[0],
    #                 colunas[4]: chroma_mean[0], colunas[5]: chroma_sd2[0], colunas[6]: chroma_mean[0], colunas[7]: chroma_sd2[0],
    #                 colunas[8]: chroma_mean[0], colunas[9]: chroma_sd2[0], colunas[10]: chroma_mean[0], colunas[11]: chroma_sd2[0],
    #                 colunas[12]: chroma_mean[0], colunas[13]: chroma_sd2[0], colunas[14]: chroma_mean[0], colunas[15]: chroma_sd2[0],
    #                 colunas[16]: chroma_mean[0], colunas[17]: chroma_sd2[0], colunas[18]: chroma_mean[0], colunas[19]: chroma_sd2[0],
    #                 colunas[20]: chroma_mean[0], colunas[21]: chroma_sd2[0], colunas[22]: chroma_mean[0], colunas[23]: chroma_sd2[0],
    #                 colunas[24]: rms_mean, colunas[25]: rms_sd2, colunas[26]: centroid_mean, colunas[27]: centroid_sd2,
    #                 colunas[28]: bandwidth_mean, colunas[29]: bandwidth_sd2, colunas[30]: roll99_mean, colunas[31]: roll99_sd2,
    #                 colunas[32]: roll01_mean, colunas[33]: roll01_sd2, colunas[34]: zcr_mean, colunas[35]: zcr_sd2,
    #                 colunas[36]: mfcc_mean[0], colunas[37]: mfcc_sd2[0], colunas[38]: mfcc_mean[1], colunas[39]: mfcc_sd2[1],
    #                 colunas[40]: mfcc_mean[2], colunas[41]: mfcc_sd2[2], colunas[42]: mfcc_mean[3], colunas[43]: mfcc_sd2[3],
    #                 colunas[44]: mfcc_mean[4], colunas[45]: mfcc_sd2[4], colunas[46]: mfcc_mean[5], colunas[47]: mfcc_sd2[5],
    #                 colunas[48]: mfcc_mean[6], colunas[49]: mfcc_sd2[6], colunas[50]: mfcc_mean[7], colunas[51]: mfcc_sd2[7],
    #                 colunas[52]: mfcc_mean[8], colunas[53]: mfcc_sd2[8], colunas[54]: mfcc_mean[9], colunas[55]: mfcc_sd2[9],
    #                 colunas[56]: mfcc_mean[10], colunas[57]: mfcc_sd2[10], colunas[58]: mfcc_mean[11], colunas[59]: mfcc_sd2[11],
    #                 colunas[60]: mfcc_mean[12], colunas[61]: mfcc_sd2[12], colunas[62]: mfcc_mean[13], colunas[63]: mfcc_sd2[13],
    #                 colunas[64]: mfcc_mean[14], colunas[65]: mfcc_sd2[14], colunas[66]: mfcc_mean[15], colunas[67]: mfcc_sd2[15],
    #                 colunas[68]: mfcc_mean[16], colunas[69]: mfcc_sd2[16], colunas[70]: mfcc_mean[17], colunas[71]: mfcc_sd2[17],
    #                 colunas[72]: mfcc_mean[18], colunas[73]: mfcc_sd2[18], colunas[74]: mfcc_mean[19], colunas[75]: mfcc_sd2[19],
    #                 colunas[76]: tempo[0], colunas[77]: y[2]}, ignore_index=True)


  return df

### Execução

In [5]:
# Lê as músicas
songs = readSongs ('all', 5, 44100, 5)


In [6]:
df = featureExtraction (songs, sr=44100, frame=1024)

In [7]:
df.head()

,Chroma mean C,Chroma sd2 C,Chroma mean C#,Chroma sd2 C#,Chroma mean D,Chroma sd2 D,Chroma mean D#,Chroma sd2 D#,Chroma mean E,Chroma sd2 E,...,MFCC17 mean,MFCC17 sd2,MFCC18 mean,MFCC18 sd2,MFCC19 mean,MFCC19 sd2,MFCC20 mean,MFCC20 sd2,tempo,Classe
0,0.394617,0.105338,0.443642,0.088150,0.481578,0.101544,0.346441,0.061531,0.370519,0.082360,...,5.050471,36.993397,-8.289567,33.612530,-2.478076,38.674591,-2.092990,34.718590,126.048018,blues
1,0.428290,0.113275,0.476215,0.106603,0.494223,0.107198,0.379621,0.076511,0.357802,0.066731,...,6.498300,29.181923,-3.217815,54.156834,3.200929,68.362213,0.460429,26.820696,123.046875,blues
2,0.357212,0.096538,0.378870,0.071623,0.492932,0.101273,0.352712,0.061301,0.321471,0.066488,...,4.673268,32.282803,-7.692242,36.558689,0.692331,38.365929,0.794478,35.058079,126.048018,blues
3,0.395143,0.086771,0.413099,0.078591,0.496900,0.105141,0.345685,0.054382,0.359460,0.067951,...,4.546683,32.767582,-5.488396,24.877171,1.453244,28.945955,-0.223831,27.314598,126.048018,blues
4,0.405898,0.121835,0.433755,0.109411,0.463097,0.106878,0.305630,0.067156,0.281570,0.061102,...,5.317678,26.526995,-4.582438,36.219418,2.514524,36.389427,-0.436270,27.635517,123.046875,blues


In [ ]:
# Extrai as features para os frame lenghts desejados
frame_lenghts = [256, 512, 1024, 2048, 4096, 8192]

dflist = []
for f in frame_lenghts:
  dflist.append(featureExtraction(songs, sr=44100, frame=f))

In [ ]:
csvnames = ['ft2/5s/256.csv', 'ft2/5s/512.csv','ft2/5s/1024.csv','ft2/5s/2048.csv','ft2/5s/4096.csv','ft2/5s/8192.csv']

for i in range(len(dflist)):
  dflist[i].to_csv(csvnames[i], sep=',', index=False, encoding='utf-8')

In [ ]:
song = songs[1]
chroma = librosa.feature.chroma_stft(y=song[0], sr=song[1], n_chroma=7)
media_linhas = np.mean(chroma, axis=1)
sd2 = np.std(chroma, axis=1)**2